In [8]:
import numpy as np
import csv
import GPy
from matplotlib import pyplot as plt
import pandas as pd
import warnings
warnings.simplefilter("error", RuntimeWarning)

In [3]:
def cnt_rmse(y, y_hat):
    return np.sqrt(((y_hat.flatten()-y.flatten())**2).mean())

In [4]:
def load_data(fname = '/home/chun/Desktop/gpr_testing/auto_mpg/auto-mpg.csv'):
    X, Y = [], []
    with open(fname, 'r') as f:
        next(f)
        for line in f:
            d = line.split(',')
            try:
                X.append([float(a) for a in d[1:8]])
#                 X.append([float(d[1])])
                Y.append([float(d[0])])
            except Exception as e:
#                 print(e)
                continue
                
    return np.array(X), np.array(Y)

X, Y = load_data()
X.shape

(392, 7)

In [5]:
# kernel = GPy.kern.RBF(input_dim=1, variance = 1., lengthscale= 1.)
kernel = GPy.kern.RBF(input_dim=7, variance = 1., lengthscale= 1.)

m = GPy.models.GPRegression(X, Y, kernel)
m.optimize(max_iters=50, messages=True)
y_hat = m.predict(X[:5,:])[0]

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i    f              |g|        
    00s01  01   5.970649e+04   6.959994e+08 
    00s10  09   5.494396e+03   5.729330e+04 
    00s19  15   1.326102e+03   2.072325e+01 
    00s59  52   1.027382e+03   6.915345e-02 
Runtime:     00s59
Optimization status: Maximum number of f evaluations reached



In [24]:
kernel = GPy.kern.RBF(input_dim = 7, variance = 1., lengthscale = 1., ARD = True)
# kernel = GPy.kern.RBF(input_dim = 7, variance = 1., lengthscale = 1.)
optimize = True
# optimize = False

data_idx = np.arange(len(X))
np.random.shuffle(data_idx)

test_size = int(np.floor(X.shape[0]/10))
split_idx = [test_size*i for i in range(1,10)]
test_idx_list = np.split(data_idx, split_idx)

train_size = 0
# train_size = 150
train_size = 50

err_list = []
for test_idx in test_idx_list:
    all_idx = np.arange(len(X))
    tmp_train_idx = np.delete(all_idx, test_idx)
    test_X = X[test_idx]
    if train_size == 0:
        new_train_idx = tmp_train_idx
    else:
        np.random.shuffle(tmp_train_idx)
        new_train_idx = tmp_train_idx[:train_size]

    new_X = X[new_train_idx]
#     print(test_X.shape)
#     print(new_X.shape)
    
    # normalize y 
    Y_mean = Y - Y[new_train_idx].mean()
    test_y = Y_mean[test_idx]
    new_y = Y_mean[new_train_idx]
    
    try:
#         m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=True)
        m = GPy.models.GPRegression(new_X, new_y, kernel, normalizer=False)
    except RuntimeWarning as e:
        print(e, '0')
#         continue
    if optimize:
        try:
            m.optimize(max_iters=50, optimizer="lbfgs")
    #         m.optimize(max_iters=100)
        except RuntimeWarning as e:
            print(e, '1')
#             continue
    try:
        y_hat = m.predict(test_X)[0]
    except RuntimeWarning as e:
        print(e, '2')
#         continue
    try:
        err = cnt_rmse(y_hat, test_y)
    except RuntimeWarning as e:
        print(e, '3')
#         continue
    
    err_list.append(err)

print(sum(err_list)/len(err_list))

7.970906457781211
